In [ ]:
# Create plot
def heatmap(ct,choose_cohort,table,table_num_rec_vis):
    # Create a new list of xlabels:
    num_visits_per_vis = table_num_rec_vis.iloc[0,:]
    visit_labels =[]

    if type(ct.cohort) == list:
        for ind in range(len(ct.visits[choose_cohort])):
            visit_labels.append("V"+ct.visits[choose_cohort][ind]+"\n"+str(num_visits_per_vis[ind]))
            if ind == len(ct.visits)-1 & (ct.studynum != "ITN084AD"):
                visit_labels[ind] = ct.visits[choose_cohort][ind]+"\n"+str(num_visits_per_vis[ind])
    else:
        for ind in range(len(ct.visits)):
            visit_labels.append("V"+ct.visits[ind]+"\n"+str(num_visits_per_vis[ind]))
            if ind == len(ct.visits)-1 & (ct.studynum != "ITN084AD"):
                visit_labels[ind] = ct.visits[ind]+"\n"+str(num_visits_per_vis[ind])
                
    title = ct.studynum + ": % of PIDs that have at least 1 Sample Collected for the Visit"
    plt.figure(figsize=(15,5))
    cmap=sns.dark_palette("#69d", reverse=True, as_cmap=True)
    fig = sns.heatmap(table,annot= True,fmt='g',cmap=cmap,vmin = 0 ,vmax = 100)
    fig.set_xticklabels(visit_labels, rotation=0)
    fig.set_yticklabels(list(table.index), rotation=0)
    fig.xaxis.tick_top()
    fig.set(xlabel='Visit',
        ylabel='Specimen Type',title=title)
    
# Get collection statistics
def pivot_table(ct,choose_cohort,rho_records,res,lv_data):
    number_of_recorded_visits = []

    # make a pivot table to get the number of recorded visits
    table_num_rec_vis = pd.pivot_table(res,values="Number of recorded visits",index = ["Sample Type"],columns=["Visit Number"])

    # re-index the table based on the ordered list of visits
    if type(ct.cohort) == list:
        table_num_rec_vis = table_num_rec_vis.reindex(ct.visits[choose_cohort],axis = 1)
    else:
        table_num_rec_vis = table_num_rec_vis.reindex(ct.visits,axis = 1)


    # make a pivot table for visualization
    table = pd.pivot_table(res,values="Percent",index = ["Sample Type"],columns=["Visit Number"])
    
    # re-index the table based on the ordered list of visits
    if type(ct.cohort) == list:
        table = table.reindex(ct.visits[choose_cohort],axis = 1)
    else:
        table = table.reindex(ct.visits,axis = 1)


    # Put NaNs in cells we are not expecting samples

    if type(ct.cohort) == list:
        try:
            cohort_exceptions = ct.exceptions[choose_cohort]
            for except_visit in cohort_exceptions:
                table.loc[except_visit[0],except_visit[1]] = np.nan
        except:
            pass
    else:
        for except_visit in ct.exceptions:
            table.loc[except_visit[0],except_visit[1]] = np.nan
    
    return table,table_num_rec_vis